In [ ]:
# 🎬 L3 — Obtención de Datos desde Archivos con Pandas

## 📍 Situación inicial
Una empresa de consultoría analiza grandes volúmenes de datos provenientes de **CSV**, **Excel** y **tablas web**.
Actualmente, el proceso es **manual**, lento y propenso a errores debido a formatos inconsistentes y datos incompletos.

El equipo ha decidido implementar **Pandas** para automatizar la carga, limpieza, transformación y exportación de datos, con el fin de garantizar calidad y consistencia.

---

## 🎯 Objetivo
Diseñar un flujo de trabajo eficiente que:
- Importe datos desde **múltiples fuentes**.
- Limpie y estandarice la información.
- Genere nuevas métricas relevantes.
- Exporte resultados listos para análisis posterior.

---

## 📂 Fuentes de datos
1. **CSV** → Dataset de películas con presupuesto, recaudación, género y ratings.
2. **Excel** → Lista simulada de películas ganadoras del Óscar.
3. **Tabla Web** → Películas con mayores recaudaciones (Wikipedia 2025).

---

## 🛠️ Tareas a realizar
1. **Carga de datos**
   - CSV → `read_csv()`
   - Excel → `read_excel()`
   - Web → `read_html()`
2. **Limpieza**
   - Manejo de valores nulos (imputar o eliminar).
   - Eliminación de duplicados.
   - Conversión de tipos de datos.
3. **Transformación**
   - Selección y renombrado de columnas.
   - Creación de métricas: ROI y Profit.
   - Ordenar por recaudación mundial.
4. **Integración**
   - Cruce con la lista de ganadoras del Óscar.
5. **Exportación**
   - CSV sin índice.
   - Excel con formato limpio.
6. **Comparación antes/después**
   - Mostrar cambios tras la limpieza.

---

## 📦 Entregables
- **Notebook en Python** con todo el código.
- **Informe técnico** con explicación de cada paso.
- **Ejemplos antes/después** de la limpieza.
- **Conclusiones** sobre la importancia de un proceso estructurado.


Para la tarea se usará un dataset de listado de películas, y generamos un archivo excel a continuación para cumplir con la tarea y como tabla se utilizará la siguiente https://es.wikipedia.org/wiki/Anexo:Pel%C3%ADculas_con_mayores_recaudaciones_(2025).

In [4]:
# genera data/oscar_winners.xlsx
from pathlib import Path
import pandas as pd

out_dir = Path("data")
out_dir.mkdir(parents=True, exist_ok=True)

df_oscar = pd.DataFrame({
    "Title": [
        "Oppenheimer","Everything Everywhere All at Once","Nomadland","Parasite","Green Book",
        "The Shape of Water","Moonlight","Spotlight","Birdman","12 Years a Slave"
    ],
    "Year": [2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014],
    "Category": ["Best Picture"]*10
})

excel_path = out_dir / "oscar_winners.xlsx"
df_oscar.to_excel(excel_path, index=False)
print("Excel generado en:", excel_path.resolve())
df_oscar.head()


Excel generado en: /content/data/oscar_winners.xlsx


,Title,Year,Category
0,Oppenheimer,2023,Best Picture
1,Everything Everywhere All at Once,2022,Best Picture
2,Nomadland,2021,Best Picture
3,Parasite,2020,Best Picture
4,Green Book,2019,Best Picture


1) Carga de datos desde distintos archivos

In [11]:
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option('display.max_columns', 60)
pd.set_option('display.width', 140)

print("== 1) Carga de datos ==")

# 1.1 CSV: IMDB 1000
url_csv = "https://raw.githubusercontent.com/justmarkham/pandas-videos/master/data/imdb_1000.csv"
df_csv = pd.read_csv(url_csv)
print("\n[1.1] CSV IMDB cargado. Shape:", df_csv.shape)
print(df_csv.head(3))

# 1.2 Excel (generado por el script anterior)
excel_path = Path("data/oscar_winners.xlsx")
if not excel_path.exists():
    raise FileNotFoundError("Falta data/oscar_winners.xlsx. Ejecuta la celda del script A).")
df_excel = pd.read_excel(excel_path)
print("\n[1.2] Excel cargado. Shape:", df_excel.shape)
print(df_excel.head())

# 1.3 Tabla web
import pandas as pd

url_web = "https://es.wikipedia.org/wiki/Anexo:Cine_en_2025"
# Busca la tabla "Películas más taquilleras de 2025"
df_web = pd.read_html(url_web, match="Películas más taquilleras")[0]

print("[1.3] Tabla web cargada. Shape:", df_web.shape)
print(df_web.head(10))



== 1) Carga de datos ==

[1.1] CSV IMDB cargado. Shape: (979, 6)
   star_rating                     title content_rating  genre  duration                                        actors_list
0          9.3  The Shawshank Redemption              R  Crime       142  [u'Tim Robbins', u'Morgan Freeman', u'Bob Gunt...
1          9.2             The Godfather              R  Crime       175    [u'Marlon Brando', u'Al Pacino', u'James Caan']
2          9.1    The Godfather: Part II              R  Crime       200  [u'Al Pacino', u'Robert De Niro', u'Robert Duv...

[1.2] Excel cargado. Shape: (10, 3)
                               Title  Year      Category
0                        Oppenheimer  2023  Best Picture
1  Everything Everywhere All at Once  2022  Best Picture
2                          Nomadland  2021  Best Picture
3                           Parasite  2020  Best Picture
4                         Green Book  2019  Best Picture
[1.3] Tabla web cargada. Shape: (10, 4)
   N.º              

2) Limpieza y estructuración de datos (sobre datos de IMDB)

In [12]:
print("== 2) Limpieza y estructuración (IMDB) ==")

# Copia para comparar antes/después
df_before = df_csv.copy()

# 2.1 Nulos: imputar numéricos y completar categóricas
print("[2.1] Nulos antes:\n", df_csv.isna().sum())

df_csv["duration"]    = pd.to_numeric(df_csv["duration"], errors="coerce")
df_csv["star_rating"] = pd.to_numeric(df_csv["star_rating"], errors="coerce")

df_csv["duration"]    = df_csv["duration"].fillna(df_csv["duration"].median())
df_csv["star_rating"] = df_csv["star_rating"].fillna(df_csv["star_rating"].median())
df_csv["content_rating"] = df_csv["content_rating"].fillna("Not Rated")
df_csv["genre"]          = df_csv["genre"].fillna("Unknown")
df_csv = df_csv.dropna(subset=["title"])

print("[2.1] Nulos después:\n", df_csv.isna().sum())

# 2.2 Duplicados por título
antes = len(df_csv)
df_csv = df_csv.drop_duplicates(subset=["title"], keep="first")
print("[2.2] Duplicados eliminados:", antes - len(df_csv))

# 2.3 Tipos de datos
print("[2.3] Tipos (muestra):\n", df_csv.dtypes.head(10))


== 2) Limpieza y estructuración (IMDB) ==
[2.1] Nulos antes:
 star_rating       0
title             0
content_rating    3
genre             0
duration          0
actors_list       0
dtype: int64
[2.1] Nulos después:
 star_rating       0
title             0
content_rating    0
genre             0
duration          0
actors_list       0
dtype: int64
[2.2] Duplicados eliminados: 4
[2.3] Tipos (muestra):
 star_rating       float64
title              object
content_rating     object
genre              object
duration            int64
actors_list        object
dtype: object


3) Transformación y optimización de datos

In [13]:
print("== 3) Transformación y optimización ==")

# 3.1 Selección de columnas relevantes
df_tr = df_csv[["title","genre","content_rating","duration","star_rating"]].copy()

# 3.2 Renombrar para traducir al español
df_tr = df_tr.rename(columns={
    "title": "titulo",
    "genre": "genero",
    "content_rating": "clasificacion",
    "duration": "duracion_min",
    "star_rating": "rating_imdb"
})

# 3.3 Ordenar por columna clave rating_imdb orden descendente
df_tr = df_tr.sort_values(by="rating_imdb", ascending=False).reset_index(drop=True)

print(df_tr.head(5))


== 3) Transformación y optimización ==
                     titulo  genero clasificacion  duracion_min  rating_imdb
0  The Shawshank Redemption   Crime             R           142          9.3
1             The Godfather   Crime             R           175          9.2
2    The Godfather: Part II   Crime             R           200          9.1
3           The Dark Knight  Action         PG-13           152          9.0
4              Pulp Fiction   Crime             R           154          8.9


4) Exportación de datos

In [14]:
print("== 4) Exportación de datos ==")
df_tr.to_csv("peliculas_imdb_limpio.csv", index=False)
df_tr.to_excel("peliculas_imdb_limpio.xlsx", index=False)
print("Listo: peliculas_imdb_limpio.csv y peliculas_imdb_limpio.xlsx")


== 4) Exportación de datos ==
Listo: peliculas_imdb_limpio.csv y peliculas_imdb_limpio.xlsx


Ejemplo de los datos antes y después de la limpieza.

In [15]:
print("ANTES (5 ejemplos):")
print(df_before.sample(5, random_state=1))

print("\nDESPUÉS (5 ejemplos):")
print(df_tr.sample(5, random_state=1))


ANTES (5 ejemplos):
     star_rating                                      title content_rating      genre  duration  \
671          7.7                                 Grindhouse              R     Action       191   
726          7.6                            Sherlock Holmes          PG-13     Action       128   
932          7.4  Harry Potter and the Order of the Phoenix          PG-13  Adventure       138   
6            8.9             The Good, the Bad and the Ugly      NOT RATED    Western       161   
954          7.4                                      X-Men          PG-13     Action       104   

                                           actors_list  
671  [u'Kurt Russell', u'Rose McGowan', u'Danny Tre...  
726  [u'Robert Downey Jr.', u'Jude Law', u'Rachel M...  
932  [u'Daniel Radcliffe', u'Emma Watson', u'Rupert...  
6    [u'Clint Eastwood', u'Eli Wallach', u'Lee Van ...  
954  [u'Patrick Stewart', u'Hugh Jackman', u'Ian Mc...  

DESPUÉS (5 ejemplos):
                    